In [9]:
from typedspark._core.column import Column


class MetaSchema(type):
    pass

class Schema(metaclass=MetaSchema):
    def __new__(cls, name: str, bases, dct):
        # cls._attributes = dir(cls)

        # initializes all uninitialied variables with a type annotation as None
        # this allows for auto-complete in Databricks notebooks (uninitialized variables
        # don't show up in auto-complete there).
        if "__annotations__" in dct.keys():
            extra = {attr: None for attr in dct["__annotations__"] if attr not in dct}
            dct = dict(dct, **extra)

        return type.__new__(cls, name, bases, dct)

class Hoi:
    a = 1

a = MetaSchema("Person", (Schema,), dict(name=Column("hoi")))

In [2]:
a.name.dtype

1

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.Builder().config("spark.ui.showConsoleProgress", "false").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/29 16:47:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/29 16:47:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/29 16:47:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
import pandas as pd

(
    spark.createDataFrame(
        pd.DataFrame(
            dict(
                name=["Jack", "John", "Jane"],
                age=[20, 30, 40],
            )
        )
    )
    .createOrReplaceTempView("person_table")
)

/Users/nanneaben/opt/anaconda3/envs/typedspark/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/nanneaben/opt/anaconda3/envs/typedspark/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [5]:
from typedspark import load_table

df, Person = load_table(spark, "person_table")

In [6]:
Person.name

Column<'name'>

In [7]:
from typedspark import Schema, Column, StructType, create_partially_filled_dataset
from pyspark.sql.types import IntegerType

class Values(Schema):
    a: Column[IntegerType]
    b: Column[IntegerType]

class Container(Schema):
    values: Column[StructType[Values]]

create_partially_filled_dataset(
    spark,
    Container,
    {
        Container.values: create_partially_filled_dataset(
            spark,
            Values,
            {Values.a: [1, 2, 3]},
        ).collect(),
    }
).createOrReplaceTempView("structtype_table")

container, ContainerSchema = load_table(spark, "structtype_table")

In [8]:
ContainerSchema.values

Column<'values'>